In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# 1. CARGA DE DATOS
df = pd.read_csv('Resultados_de_Análisis_de_Laboratorio_Suelos_en_Colombia_20260224.csv', low_memory=False)

# 2. LIMPIEZA DEL TARGET (Cultivo)
df = df[df['Cultivo'] != 'No Indica']
df = df[~df['Cultivo'].str.contains('Pasto', case=False, na=False)]

# --- NUEVO: 3. LIMPIEZA PROFUNDA DE LA COLUMNA 'RIEGO' ---
# --- 3. LIMPIEZA PROFUNDA DE 'RIEGO' Y 'TOPOGRAFIA' ---

# a. Limpieza de Riego
df['Riego'] = df['Riego'].astype(str).str.upper().str.strip()
df = df[~df['Riego'].isin(['NO INDICA', 'NAN'])]
df.loc[df['Riego'] == 'NO TIENE', 'Riego'] = 'SECANO'

# b. Limpieza de Topografía (¡NUEVO!)
df['Topografia'] = df['Topografia'].astype(str).str.upper().str.strip()
df = df[~df['Topografia'].isin(['NO INDICA', 'NAN'])]

# ---------------------------------------------------------
# ---------------------------------------------------------

# 4. SELECCIÓN DE VARIABLES
columnas_modelo = [
    'Cultivo', 'Departamento', 'Topografia', 'Riego', 
    'pH agua:suelo', 'Materia organica', 'Fósforo Bray II', 
    'Calcio intercambiable', 'Magnesio intercambiable', 'Potasio intercambiable'
]
df_limpio = df[columnas_modelo].copy()

# 5. CONVERSIÓN DE QUÍMICOS A NÚMEROS
columnas_numericas = columnas_modelo[4:] 
for col in columnas_numericas:
    df_limpio[col] = df_limpio[col].astype(str).str.replace(',', '.')
    df_limpio[col] = pd.to_numeric(df_limpio[col], errors='coerce')

df_limpio = df_limpio.dropna()

# 6. FILTRAR AL TOP 10 CULTIVOS RENTABLES
top_10_nombres = df_limpio['Cultivo'].value_counts().head(10).index
df_final = df_limpio[df_limpio['Cultivo'].isin(top_10_nombres)]

# 7. ONE-HOT ENCODING (Ahora con un 'Riego' impecable)
df_final = pd.get_dummies(df_final, columns=['Departamento', 'Topografia', 'Riego'])

# 8. ENTRENAMIENTO Y EVALUACIÓN
X = df_final.drop('Cultivo', axis=1)
y = df_final['Cultivo']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo_rf_definitivo = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
modelo_rf_definitivo.fit(X_train, y_train)

predicciones_definitivas = modelo_rf_definitivo.predict(X_test)
precision_definitiva = accuracy_score(y_test, predicciones_definitivas)

print(f"💧 Variables analizadas (limpias): {X_train.shape[1]}")
print(f"📊 ¡NUEVA Precisión global (Accuracy): {precision_definitiva * 100:.2f}%!")

# 9. SOBREESCRIBIR LOS ARCHIVOS DEL MODELO
joblib.dump(modelo_rf_definitivo, 'modelo_cultivos.pkl')
joblib.dump(X.columns.tolist(), 'columnas_modelo.pkl')
joblib.dump(modelo_rf_definitivo.classes_, 'clases_modelo.pkl')
print("\n✅ ¡Archivos .pkl actualizados con éxito!")

💧 Variables analizadas (limpias): 61
📊 ¡NUEVA Precisión global (Accuracy): 68.97%!

✅ ¡Archivos .pkl actualizados con éxito!


In [4]:
pip install -r requirements.txt


     -------------------------------------- 113.4/113.4 kB 2.2 MB/s eta 0:00:00
     -------------------------------------- 523.7/523.7 kB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 93.9/93.9 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
